In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [3]:
from sklearn import preprocessing

In [4]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [5]:
path="/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv"

In [6]:
normal_ds=pd.read_csv(path)

In [7]:
train=normal_ds.drop(normal_ds.columns[78], axis=1)

In [8]:
train.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,5.000000e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
3,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0
4,49486,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,6.666667e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0


In [9]:
#normal_ds.info()

In [10]:
#normal_ds.describe(include='all')

In [11]:
test_path="/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"

In [12]:
test=pd.read_csv(test_path)

In [13]:
test_labels=test.columns[78]

In [14]:
test=test.drop(test_labels, axis=1)

In [15]:
le_train=train.copy()
le_test=test.copy()
le = preprocessing.LabelEncoder()
for col in le_train.select_dtypes(include=['object']).columns:
    le_train[col]=le.fit_transform(le_train[col])
    le_test[col]=le.fit_transform(le_test[col])
    

In [16]:
le_train.replace([np.inf, -np.inf], np.nan, inplace=True)
le_test.replace([np.inf, -np.inf], np.nan, inplace=True)
le_train.dropna(how='any', inplace=True)
le_test.dropna(how='any', inplace=True)

In [17]:
scaler=preprocessing.StandardScaler()
s_train=scaler.fit_transform(le_train)
s_test=scaler.fit_transform(le_test)

In [18]:
s_train = np.asarray(s_train).astype(np.float32)
s_test = np.asarray(s_test).astype(np.float32)

s_train = tf.cast(s_train, tf.float32)
s_test = tf.cast(s_test, tf.float32)

In [19]:
input = tf.keras.layers.Input(shape=(78,))

In [20]:
encoder = tf.keras.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu')])(input)


In [21]:
decoder = tf.keras.Sequential([
    layers.Dense(16, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(78, activation='sigmoid')])(encoder)

In [22]:
autoencoder = tf.keras.Model(inputs=input, outputs=decoder)


In [23]:
autoencoder.compile(optimizer='adam', loss='mse', metrics = 'accuracy')


In [24]:
history = autoencoder.fit(s_train, s_train, 
          epochs=10, 
          batch_size=64,
          validation_data=(s_test, s_test),
          shuffle=True)

2022-12-07 02:07:55.444511: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-07 02:07:55.445357: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


Epoch 1/10
8274/8274 [==============================] - 5s 601us/step - loss: 0.7143 - accuracy: 0.2607 - val_loss: 0.6861 - val_accuracy: 0.2470
Epoch 2/10
8274/8274 [==============================] - 5s 559us/step - loss: 0.6655 - accuracy: 0.3507 - val_loss: 0.6740 - val_accuracy: 0.4551
Epoch 3/10
8274/8274 [==============================] - 5s 557us/step - loss: 0.6130 - accuracy: 0.4140 - val_loss: 0.6732 - val_accuracy: 0.4645
Epoch 4/10
8274/8274 [==============================] - 5s 562us/step - loss: 0.6562 - accuracy: 0.4377 - val_loss: 0.6624 - val_accuracy: 0.4903
Epoch 5/10
8274/8274 [==============================] - 5s 558us/step - loss: 0.6145 - accuracy: 0.5179 - val_loss: 0.6621 - val_accuracy: 0.4881
Epoch 6/10
8274/8274 [==============================] - 5s 564us/step - loss: 0.6979 - accuracy: 0.5154 - val_loss: 0.6594 - val_accuracy: 0.4656
Epoch 7/10
8274/8274 [==============================] - 5s 558us/step - loss: 0.6282 - accuracy: 0.5157 - val_loss: 0.6589 -